# 03 - Association Rule Mining

This notebook performs market basket analysis to find products frequently bought together.

## Objectives
- Prepare transaction data for association rule mining
- Apply Apriori algorithm
- Generate association rules
- Calculate support, confidence, and lift metrics
- Visualize product associations
